# MVD 9. cvičení

Dnešní cvičení nebude až tak obtížné. Cílem je seznámit se s HuggingFace a vyzkoušet si základní práci s BERT modelem.

## 1. část - Seznámení s HuggingFace a modelem BERT

Nainstalujte si Python knihovnu `transformers` a podívejte se na předtrénovaný [BERT model](https://huggingface.co/bert-base-uncased). Vyzkoušejte si unmasker s různými vstupy.

<br>
Pozn.: Použití BERT modelu vyžaduje zároveň PyTorch - postačí i cpu verze.

In [4]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm student of [MASK] science ")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.44494354724884033,
  'token': 3274,
  'token_str': 'computer',
  'sequence': "hello i'm student of computer science"},
 {'score': 0.19867515563964844,
  'token': 2576,
  'token_str': 'political',
  'sequence': "hello i'm student of political science"},
 {'score': 0.02690133824944496,
  'token': 2591,
  'token_str': 'social',
  'sequence': "hello i'm student of social science"},
 {'score': 0.009483722038567066,
  'token': 3019,
  'token_str': 'natural',
  'sequence': "hello i'm student of natural science"},
 {'score': 0.00781850703060627,
  'token': 6228,
  'token_str': 'mechanical',
  'sequence': "hello i'm student of mechanical science"}]

## 2. část - BERT contextualized word embeddings

BERT dokumentace obsahuje také návod jak použít tento model pro získání word embeddingů. Vyzkoušejte použití stejného slova v různém kontextu a podívejte se, jak se mění kosinova podobnost embeddingů v závislosti na kontextu daného slova.

Podívejte se na výstup tokenizeru před vstupem do BERT modelu - kolik tokenů bylo vytvořeno pro větu "Hello, this is Bert."? Zdůvodněte jejich počet.

<br>
Pozn.: Vyřešení předchozí otázky Vám pomůže zjistit, který vektor z výstupu pro cílové slovo použít.

In [14]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Hello, this is Bert."
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
output = model(**encoded_input)
print(output)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101,  7592,  1010,  2023,  2003, 14324,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0324,  0.2762, -0.2547,  ..., -0.2157,  0.1051,  0.6700],
         [ 0.2507,  0.1135,  0.0737,  ...,  0.0563,  0.9311,  0.2921],
         [-0.6409,  0.3774,  0.1627,  ..., -0.4889,  0.3352,  0.3493],
         ...,
         [ 0.7059,  0.0848,  0.5835,  ...,  0.2181,  0.4753, -0.1627],
         [ 0.4089,  0.1187, -0.6125,  ...,  0.4456, -0.2722, -0.6050],
         [ 0.7017,  0.1312, -0.4624,  ...,  0.3150, -0.5517, -0.3797]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.8061, -0.4704, -0.9495,  0.6776,  0.7262, -0.1259,  0.7468,  0.2808,
         -0.8649, -1.0000, -0.5832,  0.9513,  0.9563,  0.6321,  0.8333, -0.6898,
         -0.3962, -0.5891,  0.3375,  0.1879,  0.5585,  1.0000, -0.0234,  0.3936,
    

## Bonus - Vizualizace slovních  embeddingů

Vizualizujte slovní embeddingy - mění se jejich pozice v závislosti na kontextu tak, jak byste očekávali? Pokuste se vizualizovat i některá slova, ke kterým by se podle vás cílové slovo mělo po změně kontextu přiblížit.